### colab runtime 변경
- 코랩 런타임을 T4 GPU로 변경해주세요.
- 무료 버전 코랩을 사용할 경우, 하루 할당된 GPU가 제한되어 있어서 학습을 진행하다가 런타임이 끊길 수 있습니다.
- 로컬에 파이썬 설치가 가능하다면 로컬에서 진행하는 것을 추천드립니다.
- 로컬에서 진행할 경우엔 conda를 활용하여 가상환경을 세팅해주세요.

### mount google drive
- 코랩을 활용할 경우, 구글 드라이브에 있는 데이터를 사용하기 위해서 마운트 작업이 필요합니다.
- 로컬에서 진행할 경우엔 생략해주세요.

In [17]:
!git add "/Users/tasha/Desktop/comento/mywork/02.5_기타_모델_학습_코드.ipynb"
!git commit -m "add) :tada: detectron2 but :poop:" 
!git push origin main

fatal: '/Users/tasha/Desktop/comento/mywork/02.5_기타_모델_학습_코드.ipynb' 경로명세가 어떤 파일과도 일치하지 않습니다
현재 브랜치 main
브랜치가 'origin/main'에 맞게 업데이트된 상태입니다.

커밋하도록 정하지 않은 변경 사항:
  (무엇을 커밋할지 바꾸려면 "git add <파일>..."을 사용하십시오)
  (use "git restore <file>..." to discard changes in working directory)
	수정함:        ../../../notebooks/01.5_coco_데이터셋_구축_코드.ipynb
	수정함:        ../../../notebooks/02.5_기타_모델_학습_코드.ipynb
	수정함:        ../../../notebooks/02_yolo_모델_학습_코드.ipynb

추적하지 않는 파일:
  (커밋할 사항에 포함하려면 "git add <파일>..."을 사용하십시오)
	../../../01_과제_데이터셋_구축_코드_박다영.ipynb.zip
	../../../02_yolo_모델_학습_박다영.ipynb
	../../../02_yolo_모델_학습_박다영.ipynb.zip
	../../../2차과제_학습모델1_박다영.zip
	../../../2차과제_학습모델1_박다영/
	../../../2차과제_학습모델2_박다영 .zip
	../../../2차과제_학습모델2_박다영 /
	../../../data.yaml

커밋할 변경 사항을 추가하지 않았습니다 ("git add" 및/또는 "git commit -a"를
사용하십시오)
Everything up-to-date


In [18]:
# 파일 명 변경시.. 걍 업데이트로 하자 
! git add -u 
! git commit -m "Rename notebook files"
! git push origin main

[main ac2f181] Rename notebook files
 3 files changed, 227 insertions(+), 4290 deletions(-)
오브젝트 나열하는 중: 10, 완료.
오브젝트 개수 세는 중: 100% (10/10), 완료.
Delta compression using up to 8 threads
오브젝트 압축하는 중: 100% (6/6), 완료.
오브젝트 쓰는 중: 100% (6/6), 4.66 MiB | 2.19 MiB/s, 완료.
Total 6 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/tashapark/cctv_object_detection.git
   a34cb71..ac2f181  main -> main


### Import Libraray

In [1]:
import os
import cv2
import glob
import shutil
import warnings
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

In [2]:
#!pip install --upgrade jupyter ipywidgets

In [2]:
warnings.filterwarnings(action='ignore')

In [3]:
# 현재 실행 파일 경로 확인
os.getcwd()

'/Users/tasha/Desktop/comento/mywork/notebooks'

### detectron2 모델 설치 및 환경 설정

- 모델을 설치할 경로에 cd 명령어를 활용하여 들어가주세요

In [4]:
# 폴더 생성
#!mkdir -p /Users/tasha/Desktop/comento/mywork/models
!mkdir -p /Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN

#git ignore 추가 
#!echo "models/" >> /Users/tasha/Desktop/comento/mywork/.gitignore

In [5]:
cd /Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN

/Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN


In [6]:
# 현재 파일 경로 확인 (원하는 경로에 들어와있는지 확인)
os.getcwd()

'/Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN'

In [ ]:
# detectron2 설치
#!pip install 'git+https://github.com/facebookresearch/detectron2.git'

#### 1. 데이터셋 등록
- 먼저, COCO 형식의 데이터를 Detectron2에서 사용할 수 있도록 등록해야 합니다. 이를 위해서는 register_coco_instances 함수를 사용하여 데이터셋을 등록

In [10]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances

# COCO 데이터셋 등록
register_coco_instances("coco_train", {}, "/Users/tasha/Desktop/comento/dataset/coco_datasets/train/coco_annotations.json", "/Users/tasha/Desktop/comento/dataset/coco_datasets/train")
register_coco_instances("coco_valid", {}, "/Users/tasha/Desktop/comento/dataset/coco_datasets/valid/coco_annotations.json", "/Users/tasha/Desktop/comento/dataset/coco_datasets/valid")
register_coco_instances("coco_test", {}, "/Users/tasha/Desktop/comento/dataset/coco_datasets/test/coco_annotations.json", "/Users/tasha/Desktop/comento/dataset/coco_datasets/test")


#### 2. config.yaml 파일 설정
- 이전에 설명한 대로 Detectron2에서 훈련을 시작하려면 config.yaml 파일이 필요합니다. 
- 이 파일은 훈련에 사용될 모델과 하이퍼파라미터를 설정함

In [15]:
import yaml
import os

# 저장할 경로 (경로 명시)
yaml_path = "/Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN/config.yaml"

# YAML 파일에 저장할 내용
data = {
    "MODEL": {
        "WEIGHTS": "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl",  # 사전 훈련된 모델 경로
        "META_ARCHITECTURE": "GeneralizedRCNN",  # 모델 아키텍처 (Faster R-CNN)
        "RESNETS": {
            "DEPTH": 50  # ResNet50을 사용
        },
        "RPN": {
            "BATCH_SIZE_PER_IMAGE": 256,  # RPN 배치 크기
            "POSITIVE_FRACTION": 0.5
        },
        "ROI_HEADS": {
            "BATCH_SIZE_PER_IMAGE": 512,  # ROI 배치 크기
            "NUM_CLASSES": 1  # 클래스 개수 (car)
        },
        "INPUT": {
            "MIN_SIZE_TRAIN": 1920,  # 훈련 이미지 최소 크기
            "MAX_SIZE_TRAIN": 1920,  # 훈련 이미지 최대 크기
            "MIN_SIZE_TEST": 1920,  # 테스트 이미지 최소 크기
            "MAX_SIZE_TEST": 1920  # 테스트 이미지 최대 크기
        }
    },
    "DATASETS": {
        "TRAIN": ("coco_train",),  # 훈련 데이터셋 이름
        "TEST": ("coco_valid",)  # 검증 데이터셋 이름
    },
    "SOLVER": {
        "BASE_LR": 0.0025,  # 학습률
        "STEPS": (6000, 8000),  # 학습 스케줄러 스텝
        "MAX_ITER": 610,  # 최대 훈련 에폭 수 (10 에폭 기준)
        "IMS_PER_BATCH": 64  # 배치 크기 64
    },
    "OUTPUT_DIR": "/Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN/output"  # 훈련 결과 디렉토리
}

# YAML 파일 저장
with open(yaml_path, 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print(f"YAML 파일 생성 완료! → {yaml_path}")


YAML 파일 생성 완료! → /Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN/config.yaml


### detectron2 fasterRCNN 전이학습
- 전이학습을 진행할 때 사용되는 batch와 epoch에 대해서 정의도 같이 공부하면 좋을 것 같습니다.
- 전이학습 후에 나오는 최종모델파일은 마지막줄에 출력되는 "Results saved to yolov5/runs/train/yolov5s_results" 에 저장됩니다.

In [ ]:
#!pip install --upgrade Pillow
#!pip install torch torchvision torchaudio 
#!pip install pandas
#!pip install numpy==2.1.1 # yolov5s에 맞춤
#!pip install --upgrade scipy
#!pip install --upgrade seaborn
#!pip install tensorboard
#!pip install tensorflow
#!pip install wandb

In [8]:
import torch
print(torch.__version__)  # PyTorch 버전 확인
# MPS는 M1 GPU에서 Metal API를 사용하는 PyTorch 백엔드입니다.
print(torch.backends.mps.is_available())  # M1 GPU(Metal)가 사용 가능한지 확인

#torch는 gpu를 기본값으로 쓰도록 설정 
#device = torch.device("mps")

2.6.0
True


In [30]:
#캐시 비우기
#!rm -f /Users/tasha/Desktop/comento/dataset/datasets/labels/train.cache

In [9]:
# 완드 
import wandb
wandb.login(key="104e72e71dd69b644852d315a760589455fe70db")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/tasha/.netrc
wandb: Currently logged in as: pdy2564 (pdy2564-job-applicants) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

#### 1. 메타데이터 셋 등록

In [12]:
from detectron2.data import MetadataCatalog

# 클래스 이름 설정
MetadataCatalog.get("coco_train").set(thing_classes=["car"])
MetadataCatalog.get("coco_valid").set(thing_classes=["car"])
MetadataCatalog.get("coco_test").set(thing_classes=["car"])


namespace(name='coco_test',
          json_file='/Users/tasha/Desktop/comento/dataset/coco_datasets/test/coco_annotations.json',
          image_root='/Users/tasha/Desktop/comento/dataset/coco_datasets/test',
          evaluator_type='coco',
          thing_classes=['car'])

#### 2. 모델 학습 훈련

In [16]:
import time
import wandb
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

# WandB 설정 활성화 (entity와 project 이름 설정)
wandb.init(project="fasterRCNN_training", name="fasterRCNN_batch64_epoc10", entity="pdy2564-job-applicants")

# 시간 측정 시작
start_time = time.time()

# 설정 불러오기
cfg = get_cfg()

# config.yaml 파일 로드
cfg.merge_from_file("/Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN/config.yaml")

# 사전 훈련된 모델 로드
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(cfg.MODEL.WEIGHTS)

# 학습 설정
cfg.SOLVER.IMS_PER_BATCH = 64  # 배치 크기 설정
cfg.SOLVER.MAX_ITER = 610  # 최대 학습 스텝 설정 (에폭 수와 비례) #3888 / 64 = 60.75 ≈ 61 스텝 * 에폭 수 10
cfg.TEST.EVAL_PERIOD = 100  # 100 스텝마다 검증

# 훈련 결과 저장 디렉토리 설정
cfg.OUTPUT_DIR = "/Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN/output"

# 훈련 실행
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# 시간 측정 종료
end_time = time.time()

# 걸린 시간 출력
print(f"훈련 시간: {end_time - start_time:.2f}초")

# WandB 세션 종료
wandb.finish()


Loading config /Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN/config.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
Config '/Users/tasha/Desktop/comento/mywork/models/detectron2/fasterRCNN/config.yaml' has no VERSION. Assuming it to be compatible with latest v2.


KeyError: 'Non-existent config key: MODEL.INPUT'

#### 하이퍼파라미터 튜닝) 배치 32, 에폭 10

In [24]:
import timeit
start = timeit.default_timer()

#배치 사이즈가 16, 32, 64 정도에서 잘 작동
# 배치 32 시도, early stopping 5에폭 이상 향상이 없으면.
!python yolov5/train.py --img 224 --batch 32 --epochs 10 --data /Users/tasha/Desktop/comento/dataset/datasets/data.yaml --cfg yolov5/models/yolov5s.yaml --weights yolov5s.pt --name yolov5s_batch32_epoch10 --patience 5

end = timeit.default_timer()
print(end-start)

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Currently logged in as: pdy2564 (pdy2564-job-applicants) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=/Users/tasha/Desktop/comento/dataset/datasets/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolov5s_batch32_epoch10, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=5, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, 

#### 하이퍼파라미터 튜닝) 배치 64, 에폭 10
- 성능의 큰 차이는 없는데 시간이 확실히 줄어서 배치를 늘리는 쪽으로!

In [25]:
import timeit
start = timeit.default_timer()

# 배치 64, 에폭 10
!python yolov5/train.py --img 224 --batch 64 --epochs 10 --data /Users/tasha/Desktop/comento/dataset/datasets/data.yaml --cfg yolov5/models/yolov5s.yaml --weights yolov5s.pt --name yolov5s_batch64_epoch10 --patience 5

end = timeit.default_timer()
print(end-start)

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Currently logged in as: pdy2564 (pdy2564-job-applicants) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=/Users/tasha/Desktop/comento/dataset/datasets/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=64, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolov5s_batch64_epoch10, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=5, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, 

In [26]:
import timeit
start = timeit.default_timer()

# 배치 128, 에폭 10 --> 시도 해보고 영 아니면 에폭을 높여서.. => 비효율적
!python yolov5/train.py --img 224 --batch 128 --epochs 10 --data /Users/tasha/Desktop/comento/dataset/datasets/data.yaml --cfg yolov5/models/yolov5s.yaml --weights yolov5s.pt --name yolov5s_batch128_epoch10 --patience 5

end = timeit.default_timer()
print(end-start)

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Currently logged in as: pdy2564 (pdy2564-job-applicants) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=/Users/tasha/Desktop/comento/dataset/datasets/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=128, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolov5s_batch128_epoch10, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=5, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False

In [ ]:
import timeit
start = timeit.default_timer()

# 배치 64, 에폭 15 --> 에폭을 늘려서 시도
!python yolov5/train.py --img 224 --batch 64 --epochs 15 --data /Users/tasha/Desktop/comento/dataset/datasets/data.yaml --cfg yolov5/models/yolov5s.yaml --weights yolov5s.pt --name yolov5s_batch64_epoch15 --patience 5

end = timeit.default_timer()
print(end-start)

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Currently logged in as: pdy2564 (pdy2564-job-applicants) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=/Users/tasha/Desktop/comento/dataset/datasets/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=15, batch_size=64, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolov5s_batch64_epoch15, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=5, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, 

### yolo 모델 비교 v8

In [ ]:
#폴더 들어가기
cd /Users/tasha/Desktop/comento/mywork/models/yolov8

# 경로확인 
os.getcwd()

# 울트라틱스 임포트
!pip install ultralytics

In [ ]:
import timeit
start = timeit.default_timer()

# YOLOv8 배치 64, 에폭 10
model.train(data='/Users/tasha/Desktop/comento/dataset/datasets/data.yaml', epochs=10, batch=64, imgsz=224, name='yolov8s_batch64_epoch10')

end = timeit.default_timer()
print(f"Training time: {end - start} seconds")


### yolo 모델 비교 v11

In [ ]:
import timeit
start = timeit.default_timer()

# YOLOv11 배치 64, 에폭 10
model.train(data='/Users/tasha/Desktop/comento/dataset/datasets/data.yaml', epochs=10, batch=64, imgsz=224, name='yolov11s_batch64_epoch10')

end = timeit.default_timer()
print(f"Training time: {end - start} seconds")


In [ ]:
# 텐서보드로 학습률 상태 바로 확인 가능 
##!tensorboard --logdir yolov5/runs/train -- 비추천하심

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.19.0 at http://localhost:6006/ (Press CTRL+C to quit)


### 전이학습 결과 확인
- 1주차 과제에서 생성한 testdataset을 기반으로 모델 성능을 평가합니다.

In [ ]:
!python yolov5/val.py --img 224 --batch 60 --data datasets_yolo/datasets.yaml --weights yolov5/runs/train/yolov5s_results/weights/best.pt

val: data=datasets_yolo/datasets.yaml, weights=['yolov5/runs/train/yolov5s_results4/weights/best.pt'], batch_size=60, imgsz=224, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-362-ge8a30cf8 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/drive/MyDrive/comento/1/datasets_yolo/labels/val.cache... 399 images, 0 backgrounds, 0 corrupt: 100% 399/399 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 7/7 [00:43<00:00,  6.21s/it]
                   all        399       1072          1          1      0.995      0.901
Speed: 0.0ms pre-process, 1.1ms inference, 7.3ms NMS per imag

### 추론(inference) 진행
- 학습에 사용되지 않은 이미지를 한 장씩 넣어서 추론 결과 확인해보기
- 추론 속도와 추론 결과 이미지 확인

In [ ]:
# 필요할 경우 설치
# !pip install ultralytics

In [ ]:
import torch
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# 학습한 모델 불러오기
model = torch.hub.load('yolov5', 'custom', path='yolov5/runs/train/yolov5s_results4/weights/best.pt', source='local')
model.conf = 0.5

YOLOv5 🚀 v7.0-362-ge8a30cf8 Python-3.10.12 torch-2.4.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
# 추론을 진행할 이미지 불러오기
test_img_path = 'Enter inference image path'
img = Image.open(test_img_path)

In [ ]:
# 추론 진행
results = model(img, size=224)

/content/drive/MyDrive/comento/1/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
# 추론 결과 확인하기
print(results.pandas().xyxy[0])

# Save or display results
results.show()
plt.imshow(results.render()[0])
plt.axis('off')
plt.show()

Output hidden; open in https://colab.research.google.com to view.